In [ ]:
# --- Patient Groups: List of RML and EDF links for each patient ---
patient_groups = [
    {
        "rml_url": "RML_LINK_1",
        "edf_urls": ["EDF_LINK_1A", "EDF_LINK_1B"]
    },
    {
        "rml_url": "RML_LINK_2",
        "edf_urls": ["EDF_LINK_2A", "EDF_LINK_2B", "EDF_LINK_2C"]
    },
    # Add more patient groups as needed
]

base_dir = "/content"  # Or your working directory

# Apnea Audio Data Preparation Notebook

This notebook prepares data for apnea event detection/classification. It will:
- Download EDF and RML files for a patient
- Extract apnea events from the RML file
- Concatenate all EDFs' 'Mic' channels into one large WAV file
- Extract frame-by-frame features and labels, and append to a master CSV

## Instructions
1. Edit the download links and data directory in the first cell for each patient.
2. Run all cells in order.
3. Repeat for each patient (the master CSV will accumulate all data).

In [ ]:
# --- 2. Extract apnea events from RML ---
import xml.etree.ElementTree as ET
import csv
import os

def extract_apnea_events(xml_file_path, output_csv=None):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    namespace = {'ns': 'http://www.respironics.com/PatientStudy.xsd'}
    apnea_events = []
    for event in root.findall('.//ns:Event', namespace):
        event_family = event.get('Family')
        event_type = event.get('Type')
        if (event_family == 'Respiratory' and event_type in ['ObstructiveApnea', 'CentralApnea', 'MixedApnea']):
            start_time = float(event.get('Start'))
            duration = float(event.get('Duration'))
            end_time = start_time + duration
            apnea_events.append((event_type, start_time, end_time))
    apnea_events.sort(key=lambda x: x[1])
    if output_csv:
        with open(output_csv, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['event_type', 'start_sec', 'end_sec'])
            for event_type, start, end in apnea_events:
                writer.writerow([event_type, start, end])
        print(f"Extracted {len(apnea_events)} apnea events to {output_csv}")
    return apnea_events

apnea_csv = os.path.join(data_dir, f"{patient_id}_apnea_events.csv")
extract_apnea_events(rml_path, apnea_csv)

In [ ]:
import mne
import soundfile as sf
import numpy as np
import os

edf_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith('.edf')])
output_wav = os.path.join(data_dir, f"{patient_id}_fullmic.wav")

# Find the first EDF with a 'Mic' channel to get the sample rate
sfreq = None
for edf_path in edf_files:
    raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
    if "Mic" in raw.ch_names:
        sfreq = int(raw.info["sfreq"])
        break

if sfreq is None:
    raise RuntimeError("No EDF file with a 'Mic' channel found.")

with sf.SoundFile(output_wav, 'w', samplerate=sfreq, channels=1, subtype='PCM_16') as out_f:
    for edf_path in edf_files:
        raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
        if "Mic" not in raw.ch_names:
            print(f"❌ 'Mic' channel not found in {edf_path}!")
            continue
        mic_idx = raw.ch_names.index("Mic")
        n_samples = raw.n_times
        chunk_size = sfreq * 600  # 1 minute chunks
        for start in range(0, n_samples, chunk_size):
            stop = min(start + chunk_size, n_samples)
            mic_chunk = raw.get_data(picks=[mic_idx], start=start, stop=stop)[0]
            out_f.write(mic_chunk)
        print(f"Wrote {n_samples} samples from {edf_path}")

print(f"Saved concatenated mic channel to {output_wav}")

In [ ]:
import soundfile as sf
import numpy as np
import csv
import os
import scipy.stats
from scipy.signal import spectrogram
import librosa

def load_apnea_events(csv_path):
    events = []
    with open(csv_path, newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            events.append((float(row['start_sec']), float(row['end_sec'])))
    return events

def is_apnea(frame_start, frame_end, events):
    for start, end in events:
        if frame_end > start and frame_start < end:
            return 1  # apnea
    return 0  # non-apnea

def extract_features(frame, sr):
    # Basic features
    energy = np.mean(np.abs(frame))
    zcr = ((frame[:-1] * frame[1:]) < 0).sum() / len(frame)
    spectrum = np.abs(np.fft.rfft(frame))
    freqs = np.fft.rfftfreq(len(frame), 1/sr)
    centroid = np.sum(freqs * spectrum) / (np.sum(spectrum) + 1e-10)

    # RMS energy
    rms = np.sqrt(np.mean(frame ** 2))

    # Spectral bandwidth
    if np.sum(spectrum) > 0:
        bandwidth = np.sqrt(np.sum(((freqs - centroid) ** 2) * spectrum) / np.sum(spectrum))
    else:
        bandwidth = 0.0

    # Spectral rolloff (0.85)
    rolloff = librosa.feature.spectral_rolloff(y=frame, sr=sr, roll_percent=0.85)[0, 0]

    # Spectral flatness
    flatness = librosa.feature.spectral_flatness(y=frame)[0, 0]

    # MFCCs (first 13)
    mfccs = librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=13)
    mfccs_mean = mfccs.mean(axis=1)

    # Skewness and kurtosis
    skew = scipy.stats.skew(frame)
    kurt = scipy.stats.kurtosis(frame)

    # Spectrogram entropy (Shannon entropy of the power spectrum)
    power_spec = spectrum ** 2
    ps_norm = power_spec / (np.sum(power_spec) + 1e-10)
    entropy = -np.sum(ps_norm * np.log2(ps_norm + 1e-10))

    # Aggregate all features
    features = [energy, zcr, centroid, rms, bandwidth, rolloff, flatness, skew, kurt, entropy] + list(mfccs_mean)
    return features

apnea_events = load_apnea_events(apnea_csv)
master_csv = os.path.join(base_dir, "master_apnea.csv")
header = ['filename', 'frame_start', 'frame_end', 'energy', 'zcr', 'centroid', 'rms', 'bandwidth', 'rolloff', 'flatness', 'skew', 'kurt', 'entropy'] + [f'mfcc_{i+1}' for i in range(13)] + ['label']

write_header = not os.path.exists(master_csv) or os.stat(master_csv).st_size == 0

with sf.SoundFile(output_wav, 'r') as f:
    sr = f.samplerate
    frame_sec = 1
    frame_len = int(sr * frame_sec)
    n_frames = int(np.floor(len(f) / frame_len))
    with open(master_csv, 'a', newline='') as out_f:
        writer = csv.writer(out_f)
        if write_header:
            writer.writerow(header)
        for i in range(n_frames):
            frame = f.read(frames=frame_len, dtype='float32')
            if len(frame) < frame_len:
                break
            frame_start_sec = i * frame_sec
            frame_end_sec = frame_start_sec + frame_sec
            features = extract_features(frame, sr)
            label = is_apnea(frame_start_sec, frame_end_sec, apnea_events)
            writer.writerow([os.path.basename(output_wav), frame_start_sec, frame_end_sec] + list(features) + [label])
print(f"Appended {n_frames} frames to {master_csv}")

## Next Steps

- Change the download links and data_dir at the top for each new patient.
- Run all cells to append new data to the master CSV.
- The master CSV can now be used for classifier training in a separate notebook.

In [ ]:
# --- Loop through all patient groups and process each ---
import os

def download_file(url, out_path):
    if not os.path.exists(out_path):
        os.system(f"wget -O '{out_path}' '{url}'")
    else:
        print(f"File already exists: {out_path}")

for idx, group in enumerate(patient_groups, 1):
    patient_id = f"patient_{idx}"
    data_dir = os.path.join(base_dir, patient_id)
    os.makedirs(data_dir, exist_ok=True)
    rml_path = os.path.join(data_dir, f"{patient_id}.rml")
    download_file(group["rml_url"], rml_path)
    for i, edf_url in enumerate(group["edf_urls"], 1):
        edf_path = os.path.join(data_dir, f"no_{i}.edf")
        download_file(edf_url, edf_path)
    # After downloading, run all the data preparation steps for this patient
    # You can wrap the rest of the notebook's logic in a function and call it here for each patient
    # Example: process_patient(data_dir, patient_id)
    # (Next cells will be refactored to support this loop)

In [ ]:
# --- Function to process a single patient directory ---
import xml.etree.ElementTree as ET
import csv
import mne
import soundfile as sf
import numpy as np
import scipy.stats
from scipy.signal import spectrogram
import librosa

def extract_apnea_events(xml_file_path, output_csv=None):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    namespace = {'ns': 'http://www.respironics.com/PatientStudy.xsd'}
    apnea_events = []
    for event in root.findall('.//ns:Event', namespace):
        event_family = event.get('Family')
        event_type = event.get('Type')
        if (event_family == 'Respiratory' and event_type in ['ObstructiveApnea', 'CentralApnea', 'MixedApnea']):
            start_time = float(event.get('Start'))
            duration = float(event.get('Duration'))
            end_time = start_time + duration
            apnea_events.append((event_type, start_time, end_time))
    apnea_events.sort(key=lambda x: x[1])
    if output_csv:
        with open(output_csv, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['event_type', 'start_sec', 'end_sec'])
            for event_type, start, end in apnea_events:
                writer.writerow([event_type, start, end])
        print(f"Extracted {len(apnea_events)} apnea events to {output_csv}")
    return apnea_events

def load_apnea_events(csv_path):
    events = []
    with open(csv_path, newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            events.append((float(row['start_sec']), float(row['end_sec'])))
    return events

def is_apnea(frame_start, frame_end, events):
    for start, end in events:
        if frame_end > start and frame_start < end:
            return 1  # apnea
    return 0  # non-apnea

def extract_features(frame, sr):
    # Basic features
    energy = np.mean(np.abs(frame))
    zcr = ((frame[:-1] * frame[1:]) < 0).sum() / len(frame)
    spectrum = np.abs(np.fft.rfft(frame))
    freqs = np.fft.rfftfreq(len(frame), 1/sr)
    centroid = np.sum(freqs * spectrum) / (np.sum(spectrum) + 1e-10)

    # RMS energy
    rms = np.sqrt(np.mean(frame ** 2))

    # Spectral bandwidth
    if np.sum(spectrum) > 0:
        bandwidth = np.sqrt(np.sum(((freqs - centroid) ** 2) * spectrum) / np.sum(spectrum))
    else:
        bandwidth = 0.0

    # Spectral rolloff (0.85)
    rolloff = librosa.feature.spectral_rolloff(y=frame, sr=sr, roll_percent=0.85)[0, 0]

    # Spectral flatness
    flatness = librosa.feature.spectral_flatness(y=frame)[0, 0]

    # MFCCs (first 13)
    mfccs = librosa.feature.mfcc(y=frame, sr=sr, n_mfcc=13)
    mfccs_mean = mfccs.mean(axis=1)

    # Skewness and kurtosis
    skew = scipy.stats.skew(frame)
    kurt = scipy.stats.kurtosis(frame)

    # Spectrogram entropy (Shannon entropy of the power spectrum)
    power_spec = spectrum ** 2
    ps_norm = power_spec / (np.sum(power_spec) + 1e-10)
    entropy = -np.sum(ps_norm * np.log2(ps_norm + 1e-10))

    # Aggregate all features
    features = [energy, zcr, centroid, rms, bandwidth, rolloff, flatness, skew, kurt, entropy] + list(mfccs_mean)
    return features

def process_patient(data_dir, patient_id):
    print(f"Processing {patient_id}")
    # 1. Extract apnea events from RML
    rml_path = os.path.join(data_dir, f"{patient_id}.rml")
    apnea_csv = os.path.join(data_dir, f"{patient_id}_apnea_events.csv")
    extract_apnea_events(rml_path, apnea_csv)
    # 2. Concatenate EDFs' Mic channels
    edf_files = sorted([os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.lower().endswith('.edf')])
    output_wav = os.path.join(data_dir, f"{patient_id}_fullmic.wav")
    sfreq = None
    for edf_path in edf_files:
        raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
        if "Mic" in raw.ch_names:
            sfreq = int(raw.info["sfreq"])
            break
    if sfreq is None:
        print(f"No EDF file with a 'Mic' channel found for {patient_id}.")
        return
    with sf.SoundFile(output_wav, 'w', samplerate=sfreq, channels=1, subtype='PCM_16') as out_f:
        for edf_path in edf_files:
            raw = mne.io.read_raw_edf(edf_path, preload=False, verbose=False)
            if "Mic" not in raw.ch_names:
                print(f"❌ 'Mic' channel not found in {edf_path}!")
                continue
            mic_idx = raw.ch_names.index("Mic")
            n_samples = raw.n_times
            chunk_size = sfreq * 600  # 1 minute chunks
            for start in range(0, n_samples, chunk_size):
                stop = min(start + chunk_size, n_samples)
                mic_chunk = raw.get_data(picks=[mic_idx], start=start, stop=stop)[0]
                out_f.write(mic_chunk)
            print(f"Wrote {n_samples} samples from {edf_path}")
    print(f"Saved concatenated mic channel to {output_wav}")
    # 3. Extract features and labels, append to master CSV
    apnea_events = load_apnea_events(apnea_csv)
    master_csv = os.path.join(base_dir, "master_apnea.csv")
    header = ['filename', 'frame_start', 'frame_end', 'energy', 'zcr', 'centroid', 'rms', 'bandwidth', 'rolloff', 'flatness', 'skew', 'kurt', 'entropy'] + [f'mfcc_{i+1}' for i in range(13)] + ['label']
    write_header = not os.path.exists(master_csv) or os.stat(master_csv).st_size == 0
    with sf.SoundFile(output_wav, 'r') as f:
        sr = f.samplerate
        frame_sec = 1
        frame_len = int(sr * frame_sec)
        n_frames = int(np.floor(len(f) / frame_len))
        with open(master_csv, 'a', newline='') as out_f:
            writer = csv.writer(out_f)
            if write_header:
                writer.writerow(header)
            for i in range(n_frames):
                frame = f.read(frames=frame_len, dtype='float32')
                if len(frame) < frame_len:
                    break
                frame_start_sec = i * frame_sec
                frame_end_sec = frame_start_sec + frame_sec
                features = extract_features(frame, sr)
                label = is_apnea(frame_start_sec, frame_end_sec, apnea_events)
                writer.writerow([os.path.basename(output_wav), frame_start_sec, frame_end_sec] + list(features) + [label])
    print(f"Appended {n_frames} frames to master CSV for {patient_id}")

# --- Run all patients ---
for idx, group in enumerate(patient_groups, 1):
    patient_id = f"patient_{idx}"
    data_dir = os.path.join(base_dir, patient_id)
    print(f"\n=== Processing {patient_id} ===")
    rml_path = os.path.join(data_dir, f"{patient_id}.rml")
    # Download files (already done above)
    process_patient(data_dir, patient_id)
    print(f"Finished {patient_id}")